# Como realizar el analisis de datos en Jupyter 

[PyCon Ar en PyVideo.org](https://pyvideo.org/events/pycon-ar-2011.html)

- Explorar los datos

In [19]:
import pathlib

carpeta_datos = pathlib.Path('./datos')

In [33]:
archivos = [ elemento for elemento in carpeta_datos.glob('*/videos/*.json') ]

In [34]:
len(archivos)

237

In [35]:
{archivo.suffix for archivo in archivos }

{'.json'}

In [31]:
archivos[0].suffix

''

- Importar los datos con pandas

In [36]:
import pandas

In [37]:
pandas.read_json(archivos[0])

,description,language,recorded,speakers,thumbnail_url,title,videos
0,Alan Runyan y su charla en el auditorio princi...,eng,2011-09-23,Alan Runyan,https://i.ytimg.com/vi/TNnU2oqPxUI/hqdefault.jpg,Plone,"{'type': 'youtube', 'url': 'https://www.youtub..."


- Realizar graficos